# DSCI 100 Group Project Report: The relationship between various socioeconomic factors and life expectancy at birth
##### Group 141: Matilde Galantino, Jacob Guglielmin, Phoenix Jin, Amy Wong 

## Introduction
### Background 

Life expectancy, a measurement of the mean age of death in a population, is a crucial metric in assessing the wellbeing of populations, and as such, is of pertinence in both medical and socioeconomic research. Life expectancy has risen drastically in industrialized countries since the 19th century due to improvements in technology and healthcare, from 66.8 years in 2000 to 73.4 years in 2019 (World Health Organization, 2019). However, this increase is not truly equal - countries with lower human development have consistently reported lower life expectancies, leading to a large discrepancy in health worldwide (Roser et al., 2013). It is therefore important to determine what impedes or enhances the life expectancy of a population in order to improve societal conditions around the world.

### Question 

How do CO<sub>2</sub> emissions per capita, mean years of schooling, gross national income, and poverty rate affect life expectancy for both less developed and more developed countries, and how can less developed countries use this information to increase their life expectancy?

### Dataset 

We will use two datasets from the United Nations Development Programme and the World Bank; the one from the United Nations Development Programme contains data on CO<sub>2</sub> emissions per capita, mean years of schooling, gross national income, and life expectancy at birth by country and year, and the dataset from the World Bank contains data on poverty rate by country and year.

## Expected outcomes: 
We expect to find relationships between all four of our selected socioeconomic factors and life expectancy at birth. 

1. CO<sub>2</sub> concentrations have increased in the atmosphere over time due to human activity. Several decades of research have indicated that rising air pollution threatens local ecosystems and human health (Balakrishnan et al., 2018). As such, we expect to find a negative relationship between CO<sub>2</sub> emissions and life expectancy.

2. Education has an important role in shaping society, and so will be investigated for contributions to life expectancy. Past research has indicated that higher levels of schooling lead to higher life expectancies due to more informed decisions and healthier lifestyles (Blackburn and Cipriani, 2002). Therefore, we expect to see a positive relationship between mean years of schooling and life expectancy.

3. The gross national income represents the total financial output of a country. At a national level, a higher GNI allows more spending for public systems such as improved healthcare and sanitation. Therefore, we expect a positive relationship between GNI and life expectancy. 

4. Lastly, the poverty rate represents the percentage of the country's population who has less than $6.85 to spend per day, in 2017 PPP dollars. Financial poverty very often leads to social effects such as poor living conditions and limited access to housing or healthcare. We expect a negative relationship between poverty ratio and life expectancy at birth.


## Methods: 
1. We will combine the two datasets that we selected. 

2. We will use the columns for the four variables as predictors, which are carbon dioxide emissions per capita, mean years of schooling, gross national income per capita, poverty rate, to predict life expectancy at birth

3. We will combine countries with 'Very High' and 'High' human development indices into a 'More Developed' category, and combine countries with 'Medium' and 'Low' indices into a 'Less Developed' category.

4. We will perform a multivariable regression with our four factors in order to predict life expectancy at birth.

5. We will use four scatterplots to visualize our results for each of our factors with their regression lines overlaid on their respective plots. We will also create a table displaying the RMSPE of each of our regressions, to compare the predictive power of each more easily.

##### **Run the following cell before continuing further - set** `need_install = TRUE` **if any of tidyverse, tidymodels, or cowplot are not installed on your system:**

In [ ]:
need_install = FALSE

if (need_install) install.packages(c("tidyverse", "tidymodels", "cowplot"))

# Import libraries
library(tidyverse)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6)

### Read datasets from the web:

##### We are reading these data from GitHub, as we downloaded the data manually and uploaded the data we require since much of the data came packaged in a .zip file rather than available as a raw .csv. In addition, we can ensure that these links remain available, unlike the links of the original publishers.

In [ ]:
# Download data - (data orginally from United Nations Development Programme)
hdi_composite_data_raw <- read_csv("https://raw.githubusercontent.com/phoenixjin8/dsci100-group-project/main/data/hdr_composite_index.csv", show_col_types = FALSE)

#Print the table number and data source 
comment(hdi_composite_data_raw)<-"Table 1: Data from United Nations Development Programme"

comment(hdi_composite_data_raw)
hdi_composite_data_raw 

In [ ]:
# Download data - (data orginally from World Bank) 
poverty_rate_data_raw <- read_csv("https://raw.githubusercontent.com/phoenixjin8/dsci100-group-project/main/data/poverty_rate.csv", skip = 4, show_col_types = FALSE)

#Print the table number and data source 
comment(poverty_rate_data_raw) <- "Table 2: Data from World Bank"

comment(poverty_rate_data_raw)
poverty_rate_data_raw

### Clean and wrangle data into a tidy format:

In [ ]:
# Tidy poverty rate data
poverty_rate_data <- poverty_rate_data_raw
poverty_rate_data <- poverty_rate_data |>
    select(-country_name, -indicator_name, -indicator_code, -development) |>
    pivot_longer(c(-country_code), names_to = "year", values_to = "poverty_rate") |>
    filter(!is.na(poverty_rate))

# Tidy HDI data
# Eliminate irrelevant columns
hdi_composite_data_relevant <- hdi_composite_data_raw |>
    select(-country, -hdicode, -region, -hdi_rank_2021)
# We will write to a new dataframe rather than edit the existing one since the raw data is horrific to work with
# Extract CO2 emissions data and country codes
hdi_composite_data <- hdi_composite_data_relevant |>
    select(iso3, co2_prod_1990:co2_prod_2021) |>
    # Remove all information from column name except the year (change each column name to the last 4 characters of itself, ignore "iso3")
    rename_with(function(x){ifelse(x == "iso3", "iso3", substring(x, nchar(x) - 3))}) |>
    pivot_longer(-iso3, names_to = "year", values_to = "co2_per_capita")
# Extract mean years of schooling data
hdi_composite_data <- merge(hdi_composite_data,
                            (hdi_composite_data_relevant |>
                             select(iso3, mys_1990:mys_2021) |>
                             rename_with(function(x){ifelse(x == "iso3", "iso3", substring(x, nchar(x) - 3))}) |>
                             pivot_longer(-iso3, names_to = "year", values_to = "mean_schooling")))
# Extract GNI per capita data
hdi_composite_data <- merge(hdi_composite_data,
                            (hdi_composite_data_relevant |>
                             select(iso3, gnipc_1990:gnipc_2021) |>
                             rename_with(function(x){ifelse(x == "iso3", "iso3", substring(x, nchar(x) - 3))}) |>
                             pivot_longer(-iso3, names_to = "year", values_to = "gni_per_capita")))
# Extract life expectancy data
hdi_composite_data <- merge(hdi_composite_data,
                            (hdi_composite_data_relevant |>
                             select(iso3, le_1990:le_2021) |>
                             rename_with(function(x){ifelse(x == "iso3", "iso3", substring(x, nchar(x) - 3))}) |>
                             pivot_longer(-iso3, names_to = "year", values_to = "life_expectancy")))

# Merge HDI data with poverty rate data and convert year to a number
tidy_data <- merge(poverty_rate_data, hdi_composite_data, by.x = c("country_code", "year"), by.y = c("iso3", "year"), all = TRUE) |>
    mutate(year = as.numeric(year))

# Add development rating
tidy_data <- left_join(tidy_data, select(poverty_rate_data_raw, country_code, development), by = "country_code")    

# Consolidate Low and Medium development into "Less Developed", High and Very High into "More Developed"
tidy_data <- filter(tidy_data, !is.na(development)) |>
    mutate(development = ifelse(development == "Low" | development == "Medium", "Less Developed", "More Developed")) |>
    mutate(development = as.factor(development))

# Transform the data so that we can use a linear regression
tidy_data <- tidy_data |>
    mutate(log_co2_per_capita = log(co2_per_capita)) |>
    mutate(log_gni_per_capita = log(gni_per_capita))

# Reorder the columns to make sense
tidy_data <- tidy_data[,c("country_code", "development", "year", "life_expectancy", "gni_per_capita", "log_gni_per_capita", "poverty_rate", "mean_schooling", "co2_per_capita", "log_co2_per_capita")] |>
    
# Remove the 4 data points that are WAY out the bottom of the life expectancy chart - these appear to be
# substantial departures from normal for their countries (Rwanda and South Sudan) and are directly between
# much higher values, so we are assuming these to be data collection errors
    filter(life_expectancy > 25) |>

# Remove every row with no life expectancy value, since everything is based on those values
    filter(!is.na(life_expectancy))

# Reorder the development category so that legends are in a reasonable order
tidy_data$development <- factor(tidy_data$development, levels = c("More Developed", "Less Developed"))

#Print table number and data name 
comment(tidy_data) <- "Table 3: Tidy Data"

comment(tidy_data)
tidy_data

### Split data to produce training/testing data:

In [ ]:
set.seed(141)

# Split data into training and testing groups, stratified by life expectancy
data_split <- initial_split(tidy_data, prop = 0.75, strata = life_expectancy)
training_data <- training(data_split)
testing_data <- testing(data_split)

## Exploratory data analysis

#### Create visualizations for the data:

In [ ]:
# Select the columns that contain data we plan to model
data_cols <- select(training_data, life_expectancy:co2_per_capita)

# Count number of datapoints per column by development category
development_groups_counts <- select(training_data, -country_code, -year) |>
    group_by(development) |>
    summarize_all(list(~ sum(!is.na(.)))) |>
    add_row(mutate(as_tibble_row(colSums(!is.na(data_cols))), development = "Combined"), .before = 1)

#Add table number and table name  
comment(development_groups_counts) <- "Table 4: Developed groups counts"

In [ ]:
# Take mean of each column by development category
development_groups_means <- select(training_data, -country_code, -year) |>
    group_by(development) |>
    summarize_all(list(~ sum(., na.rm = TRUE)/sum(!is.na(.)))) |>
    add_row(mutate(as_tibble_row(colMeans(data_cols, na.rm = TRUE)), development = "Combined"), .before = 1)

#Add table number and table name 
comment(development_groups_means) <- "Table 5: Developed groups means"

In [ ]:
# Create plot of life expectancy vs. GNI
options(repr.plot.width = 14, repr.plot.height = 9)
gni_plot <- training_data |>
    ggplot(aes(x = log_gni_per_capita, y = life_expectancy, colour = development)) +
    geom_point(alpha = 0.5) +
    labs(x = "GNI (per capita)", y = "Life Expectancy", colour = "Development") +
    theme(text = element_text(size = 19)) +
    geom_smooth(method = "lm", se = TRUE, formula = y ~ x, color = "black")

In [ ]:
# Create plot of life expectancy vs. poverty rate
poverty_plot <- training_data |>
    ggplot(aes(x = poverty_rate, y = life_expectancy, colour = development)) +
    geom_point(alpha = 0.5) +
    labs(x = "Poverty Rate (% of population living at <$6.85/day, 2017 PPP)", y = "Life Expectancy", colour = "Development") +
    theme(text = element_text(size = 19)) +
    geom_smooth(method = "lm", se = TRUE, formula = y ~ x, color = "black")

In [ ]:
# Create plot of life expectancy vs. mean schooling
schooling_plot <- training_data |>
    ggplot(aes(x = mean_schooling, y = life_expectancy, colour = development)) +
    geom_point(alpha = 0.5) +
    labs(x = "Mean Schooling (years)", y = "Life Expectancy", colour = "Development") +
    theme(text = element_text(size = 19)) +
    geom_smooth(method = "lm", se = TRUE, formula = y ~ x, color = "black")

In [ ]:
# Create plot of life expectancy vs. CO2 emissions
co2_plot <- training_data |>
    ggplot(aes(x = log_co2_per_capita, y = life_expectancy, colour = development)) +
    geom_point(alpha = 0.5) +
    labs(x = "CO2 Emissions (tonnes per capita)", y = "Life Expectancy", colour = "Development") +
    theme(text = element_text(size = 19)) +
    geom_smooth(method = "lm", se = TRUE, formula = y ~ x, color = "black")

#### Number of data points per development rating for each variable:

##### This information allows us to identify areas where we may not have as large a pool of data to draw conclusions from, and keeps us aware of this as a source of error throughout our work.

In [ ]:
comment(development_groups_counts)
development_groups_counts

#### Mean of the data points from each development rating for each variable:

##### This information allows us to see, at-a-glance, overall trends of the change in each variable between development ratings and gives us an idea of the scale of the data we are going to be working with.

In [ ]:
comment(development_groups_means)
development_groups_means

#### Plot life expectancy against GNI per capita:

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 9)
gni_plot + ggtitle("Figure 1 - Life Expectancy vs. GNI per capita")

#### Relationship between life expectancy and GNI per capita:
##### As seen in the scatterplot, more developed countries and less devloped countries are very distinct. For more developed countries, as the gross national income increases (x value increases), the life expectancy also increases (y value increases). In comparision, for less developed countries, most of the data are on the left side of the graph with relatively low gross national income; the gross national income of less developed countries does not increase much. In addition, more developed countries covers a greater range of and have higher gross national income than less developed countries. Therefore, we conclude that there is a weak positive relationship between gross national income and life expectancy; as gross national income increases, life expectanvy also increases. 

#### Plot life expectancy against poverty rate:

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 9)
poverty_plot + ggtitle("Figure 2 - Life Expectancy vs. Poverty Rate")

#### Relationship between life expectancy and Poverty Rate:
##### As seen in the scatterplot, more developed countries and less developed countries are very distinct; majority data for more developed countries are on the left side of the graph with low poverty rate and high life expectany, while the majority data for less developed countries are on the right side of the graph with high poverty rate and low life expectancy. Also, there is a linear negative relationship between poverty rate and life expectancy; as poverty rate increases, life expectancy decreases. Finally, poverty rate is a good predictor because it shows relatively linear relationships and a clear difference between more and less developed countries. 

#### Plot life expectancy against the mean years of schooling:

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 9)
schooling_plot + ggtitle("Figure 3 - Life Expectancy vs. Mean Years of Schooling")

#### Relationship between life expectancy and mean years of schooling:
##### As seen in the graph, more developed countries and less developed countries are very different; more developed countries have higher mean years of schooling and higher life expectancy, while less developed countries have lower mean years of schooling and lower life expectancy. Also, there is a linear positve relationship between mean years of schooling and life expectancy; as mean years of schooling increases, life expectancy increases. Finally, mean years of schooling is a good predictor because it shows a linear positive relationship and a clear difference between higher and lower developed countries. 

#### Plot life expectancy against CO<sub>2</sub> emissions (tonnes per capita):

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 9)
co2_plot + ggtitle("Figure 4 - Life Expectancy vs. CO2 Emissions")

#### Relationship between life expectancy and CO2 Emissions: 
##### As seen in the graph, more developed countries and less developed countries are very different; more developed countries generally have higher CO2 emissions and higher life expectancy, while less developed countries have lower CO2 Emissions and lower life expectancy. Majority data for less developed countries are on the left of the graph and do not increase much, while more developed countries cover a greater range of CO2 emissions. Also, there is a weak positive relationship between CO2 emissions and life expectancy; as CO2 emissions increases, life expectancy shows a slight decrease for more developed countries. 

#### Data analysis
##### Build a model

In [ ]:
#Set seed to make unbiased and reproducible analysis
set.seed(141)

lm_spec <- linear_reg() |>
    set_engine("lm") |>
    set_mode("regression")

#Create recipe to build our model
lm_recipe <- recipe(life_expectancy ~ log_gni_per_capita + poverty_rate + mean_schooling + log_co2_per_capita, data = training_data)

#Fit linear models to training data
lm_fit <- workflow() |>
       add_recipe(lm_recipe) |>
       add_model(lm_spec) |>
       fit(data = training_data)

lm_fit

In [ ]:
#check p value 
print("Table 6: results of lm_fit") 
tidy(lm_fit)


##### interpretation of coefficients and p.value 

As seen in the lm_fit, the coefficients of log_gni_per_capita and mean_schooling are positive, and the coefficients of log_co2_per_capita and poverty_rate are negative. Thus, log_gni_per_capita and mean_schooling contribute positively to life expectancy at birth, and log_co2_per_capita and poverty_rate contribute negatively to life expectancy at bith. Even though the co2_per_capita has a positive relationship with life expectancy in figure 4, when considering all socioeconomic factors, co2 emissions have a negative impact on life expectancy. 

Moving on to the p values as shown in table 6, all p values are much lower than 0.05. Thus, we reject the null hypothesis that the slope of each socioeconomic factors is zero, which proves that there is a relationship between these socioeconomic factors and life expectancy at birth. 

##### Perform the data analysis

In [ ]:
#remove rows that missing data for poverty rate column 
new_testing_data <- testing_data|>
    filter(!is.na(poverty_rate))

#Predicting life expectancy of data from the testing data set
lm_results<- predict(lm_fit, new_testing_data)|>
    bind_cols(new_testing_data)

#Add table number and table name 
comment(lm_results) <- "Table 7: testing data with predicted life expectancy"

comment(lm_results)
lm_results

##### Check RMSPE 

In [ ]:
#Get rmspe based on testing data 
rmspe_testing <- lm_results|>
    metrics(truth=life_expectancy, estimate=.pred)|>
    filter(.metric=="rmse")|>
    select(.estimate)|>
    pull()

rmspe_testing

## Discussion (Results) 
- We found that life expectancy has a postive relationship with gross national income, mean years of schooling, and CO2 emissions, but a negative relationship with poverty rate. Also, more developed countries have higher life expectancy than less developed countries. 
- The majority of relationships match our predictions; we expected that life expectancy has a positve relationship with gross national income and mean years of schooling, and a negative relationship with poverty rate and CO2 emissions. However, according to our graphs, CO2 emission doesn't match our expectations, which has a positive relationship with life expectancy. 
- The main impact of these findings is to identify socioeconomic factors that contribute to life expectancy at birth and determine how gross national income, mean years of schooling, CO2 emissions, and poverty affect life expectancy at birth.
- The future questions is how humanitarian efforts may target specific factors such as education that would have the greatest impact on extending life expectancy in less developed countries. In addition, our findings may inform other avenues of research, such as how the model for the social cost of carbon could be improved by including life expectancy as a factor.


## References

Blackburn, K., & Cipriani, G.P. (2002). A model of longevity, fertility and growth. <i>Journal of Economic Dynamics and Control, 26,</i> 187-204.

Hill, T. D., Jorgenson, A. K., Ore, P., Balistreri, K. S., & Clark, B. (2018). Air quality and life expectancy in the United States: An analysis of the moderating effect of income inequality. <i>SSM - population health, 7,</i> 100346. https://doi.org/10.1016/j.ssmph.2018.100346

Human Development Reports Data Center. (n.d.). https://hdr.undp.org/data-center 

Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population) | Data. (n.d.). https://data.worldbank.org/indicator/SI.POV.UMIC

Roser, M., Ortiz-Ospina, E., & Ritchie, H. (2013). <i>Life Expectancy</i>. Retrieved from https://ourworldindata.org/life-expectancy

World Health Organization. (2020). <i>GHE: Life expectancy and healthy life expectancy</i>. Retrieved from https://www.who.int/data/gho/data/themes/mortality-and-global-health-estimates/ghe-life-expectancy-and-healthy-life-expectancy